In [48]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
base_dir = ""
df_train = pd.read_csv(os.path.join(base_dir,"pro_train.csv"))
df_test = pd.read_csv(os.path.join(base_dir,"pro_test.csv"))
df_train.head(5)

,Unnamed: 0,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit,Stay
0,231676,0,7,1,4,2,1,2,2.0,8.0,0,1,4,4,6247.0,2
1,166821,0,7,1,2,3,3,2,1.0,2.0,0,1,6,6,8000.0,10
2,70566,1,2,1,2,3,2,3,3.0,7.0,0,2,3,2,4987.0,1
3,197982,1,2,1,2,2,1,3,3.0,14.0,1,1,3,6,7210.0,6
4,280389,3,13,1,4,3,2,1,2.0,8.0,2,1,2,4,3178.0,5


In [49]:
df_train = df_train.drop(["Unnamed: 0"], axis = 1)
df_test = df_test.drop(["Unnamed: 0"], axis=1)

df_y = df_train.Stay
df_train = df_train.drop("Stay", axis = 1).dropna()

df_test_y = df_test.Stay
df_test = df_test.drop("Stay", axis=1).dropna()

In [50]:
from sklearn.preprocessing import StandardScaler

num_attribs = ["Available Extra Rooms in Hospital", "Bed Grade", "Visitors with Patient", "Admission_Deposit"]
cat_attribs = ["Hospital_type_code", "City_Code_Hospital", "Hospital_region_code", "Department", "Ward_Type", "Ward_Facility_Code", "City_Code_Patient", "Type of Admission"]

scaler = StandardScaler()

df_train[num_attribs] = scaler.fit_transform(df_train[num_attribs])
df_test[num_attribs] = scaler.fit_transform(df_test[num_attribs])

In [6]:
%pip install pytorch-tabnet

In [65]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from pytorch_tabnet.tab_model import TabNetClassifier

cat_idxs = [ i for i, f in enumerate(df_train.columns) if f in cat_attribs]
cat_dims = [len((df_train[cat]+df_test[cat]).unique()) for cat in cat_attribs]

# https://pypi.org/project/pytorch-tabnet/
model = TabNetClassifier(n_d=12, n_a=12, cat_idxs=cat_idxs, cat_dims=cat_dims, n_steps=5)

Device used : cpu


In [66]:
model.fit(
    X_train=df_train.values, y_train=df_y.values,
    eval_set=[(df_train.values, df_y.values), (df_test.values, df_test_y.values)],
    eval_metric=["accuracy"],
    eval_name=["train", "test"],
    max_epochs=200, patience=100,
    batch_size=5000, virtual_batch_size=256
)

epoch 0  | loss: 2.05666 | train_accuracy: 0.33511 | test_accuracy: 0.33524 |  0:00:38s
epoch 1  | loss: 1.71435 | train_accuracy: 0.34915 | test_accuracy: 0.34854 |  0:01:16s
epoch 2  | loss: 1.68351 | train_accuracy: 0.35816 | test_accuracy: 0.35701 |  0:01:54s
epoch 3  | loss: 1.66891 | train_accuracy: 0.36431 | test_accuracy: 0.36366 |  0:02:35s
epoch 4  | loss: 1.65839 | train_accuracy: 0.37041 | test_accuracy: 0.37118 |  0:03:17s
epoch 5  | loss: 1.65044 | train_accuracy: 0.37248 | test_accuracy: 0.37305 |  0:03:56s
epoch 6  | loss: 1.64993 | train_accuracy: 0.37521 | test_accuracy: 0.37608 |  0:04:36s
epoch 7  | loss: 1.63936 | train_accuracy: 0.37675 | test_accuracy: 0.37801 |  0:05:12s
epoch 8  | loss: 1.63424 | train_accuracy: 0.38091 | test_accuracy: 0.37995 |  0:05:49s
epoch 9  | loss: 1.62227 | train_accuracy: 0.39002 | test_accuracy: 0.39224 |  0:06:27s
epoch 10 | loss: 1.5952  | train_accuracy: 0.39549 | test_accuracy: 0.39406 |  0:07:07s
epoch 11 | loss: 1.58546 | train

In [ ]:
[
    {
        epochs=200,
        batch_size=1000,
        acc=42,
        test_acc=42,
    },
    {
        n_d=12,
        n_shared=4,
        epochs=200,
        batch_size=5000,
        n_steps=3,
        acc=42,
        test_acc=42,
    }
]